In [1]:
from keras.utils import to_categorical
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing import image
import os
import sys
import random
import shutil
import numpy as np

module_path = os.path.abspath(os.path.join('../homework'))
if module_path not in sys.path:
    sys.path.append(module_path)

from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數

# 預防錯誤： OMP: Error #15: Initializing libiomp5.dylib, but found libiomp5.dylib already initialized.
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

Using TensorFlow backend.


In [2]:
# 資料路徑
DATASET_PATH  = 'images'

# 原始訓練集目錄
TRAIN_SET_FOLDER = "train"

# 測試集目錄
TEST_SET_FOLDER = "test"

# 執行訓練集、驗證集目錄
RUNNING_TRAIN_SET_FOLDER = "_train"
RUNNING_VALID_SET_FOLDER = "_val"

# F照和O照目錄
FXXX_IMAGE_FOLDER = "fuck"
OXXX_IMAGE_FOLDER = "oral"

# 驗證資料佔全部訓練資料的比例
RATIO_OF_VALIDATION_SET = 0.1

# 影像大小
IMAGE_SIZE = (224, 224)

# 影像類別數
NUM_CLASSES = 2

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 4

# 凍結網路層數
FREEZE_LAYERS = 2

# Epoch 數
NUM_EPOCHS = 20

# 模型輸出儲存的檔案
WEIGHTS_FINAL = 'model-resnet50-final.h5'

In [3]:
# 刪除既有的執行訓練集、驗證集目錄

if os.path.isdir(os.path.join(DATASET_PATH, RUNNING_TRAIN_SET_FOLDER)):
    shutil.rmtree(os.path.join(DATASET_PATH, RUNNING_TRAIN_SET_FOLDER)) 
    
if os.path.isdir(os.path.join(DATASET_PATH, RUNNING_VALID_SET_FOLDER)):
    shutil.rmtree(os.path.join(DATASET_PATH, RUNNING_VALID_SET_FOLDER)) 

In [4]:
# 將原始訓練集裡的F照和O照分成訓練及驗證集
for img_folder in [FXXX_IMAGE_FOLDER, OXXX_IMAGE_FOLDER]:
    img_full_path = os.path.join(DATASET_PATH, TRAIN_SET_FOLDER, img_folder)
    img_train_path = os.path.join(DATASET_PATH, RUNNING_TRAIN_SET_FOLDER, img_folder)
    img_valid_path = os.path.join(DATASET_PATH, RUNNING_VALID_SET_FOLDER, img_folder)

    img_files = [f for f in os.listdir(img_full_path) if os.path.isfile(os.path.join(img_full_path, f))]
    valid_images = random.sample(img_files,  int(len(img_files) * RATIO_OF_VALIDATION_SET))
    train_images = [f for f in img_files if f not in valid_images]

    os.makedirs(img_train_path)    
    os.makedirs(img_valid_path)

    for img in train_images:
        shutil.copy(os.path.join(img_full_path, img), img_train_path)
    for img in valid_images:
        shutil.copy(os.path.join(img_full_path, img), img_valid_path) 

In [5]:
# 透過 data augmentation 產生訓練與驗證用的影像資料
train_datagen = image.ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_batches = train_datagen.flow_from_directory(os.path.join(DATASET_PATH, RUNNING_TRAIN_SET_FOLDER),
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_datagen = image.ImageDataGenerator()
valid_batches = valid_datagen.flow_from_directory(os.path.join(DATASET_PATH, RUNNING_VALID_SET_FOLDER),
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

# 輸出各類別的索引值
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

Found 156 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Class #0 = fuck
Class #1 = oral


In [6]:
# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

# 輸出整個網路結構
print(net_final.summary())

# 訓練模型
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

# 儲存訓練好的模型
net_final.save(WEIGHTS_FINAL)

/Users/ken/virtualenv/default/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
W0823 15:10:55.176229 4784797120 deprecation.py:506] From /Users/ken/virtualenv/default/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
______________________________________________________________________________________________

Epoch 1/20
39/39 [==============================] - 147s 4s/step - loss: 1.3412 - acc: 0.6282 - val_loss: 0.6347 - val_acc: 0.7500
Epoch 2/20
39/39 [==============================] - 116s 3s/step - loss: 0.9469 - acc: 0.6410 - val_loss: 0.3829 - val_acc: 0.8125
Epoch 3/20
39/39 [==============================] - 120s 3s/step - loss: 0.7940 - acc: 0.7372 - val_loss: 0.2779 - val_acc: 0.8125
Epoch 4/20
39/39 [==============================] - 116s 3s/step - loss: 0.6852 - acc: 0.7564 - val_loss: 0.2407 - val_acc: 0.8750
Epoch 5/20
39/39 [==============================] - 121s 3s/step - loss: 0.4975 - acc: 0.8141 - val_loss: 0.1473 - val_acc: 0.8750
Epoch 6/20
39/39 [==============================] - 118s 3s/step - loss: 0.6045 - acc: 0.8013 - val_loss: 0.1776 - val_acc: 0.8750
Epoch 7/20
39/39 [==============================] - 120s 3s/step - loss: 0.5079 - acc: 0.8590 - val_loss: 0.1306 - val_acc: 0.9375
Epoch 8/20
39/39 [==============================] - 118s 3s/step - loss: 0.3446 - a

In [12]:
# 從參數讀取圖檔路徑
img_path = os.path.join(DATASET_PATH, TEST_SET_FOLDER)
files = os.listdir(img_path)
list.sort(files)

# 載入訓練好的模型
net = load_model('model-resnet50-final.h5')

cls_list = [FXXX_IMAGE_FOLDER, OXXX_IMAGE_FOLDER]

# 辨識每一張圖
for f in files:
    img = image.load_img(os.path.join(img_path, f), target_size=(224, 224))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    pred = net.predict(x)[0]
    top_inds = pred.argsort()[::-1][:5]
    print(f)
    for i in top_inds:
        print('    {:.3f}  {}'.format(pred[i], cls_list[i]))

001.jpg
    0.577  fuck
    0.423  oral
002.jpg
    0.803  oral
    0.197  fuck
003.jpg
    0.660  oral
    0.340  fuck
004.jpg
    0.965  fuck
    0.035  oral
005.jpg
    0.729  oral
    0.271  fuck
006.jpg
    0.828  oral
    0.172  fuck
007.jpg
    0.802  fuck
    0.198  oral
008.jpg
    1.000  oral
    0.000  fuck
009.jpg
    0.623  oral
    0.377  fuck
010.jpg
    1.000  oral
    0.000  fuck


### Result
#### 2019-08-23：60%
- Train：20 epochs、loss: 0.1564 - acc: 0.9487 - val_loss: 0.0329 - val_acc: 1.0000
- Correct：001、004、006、008、009、1.0
- Incorect：002、003、005、007